# Create Outlook events during PT sessions

Read Google Calendar `Treino` and finds all PT sessions, then creates an event in Outlook caldendar at the same time.

Finding events is done by searching for `Tania` or `PT` in the event name.

The Outlook event is created starting 15min, and last for 1h:45min.

In [1]:
import os 
import datetime as dt

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

from calendars import OutlookCalendar

In [2]:
dt.datetime.now()

datetime.datetime(2024, 5, 28, 15, 16, 51, 172528)

In [11]:
DRY_RUN = True
START_DATE = dt.datetime(2024, 5, 1)
START_DATE = dt.datetime.now()
SCOPES = ['https://www.googleapis.com/auth/calendar']
OUTLOOK_CALENDAR = OutlookCalendar()

In [8]:
def _get_credentials():
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', 
                                                      SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    return creds

creds = _get_credentials()

In [9]:
service = build('calendar', 'v3', credentials=creds)

In [10]:
from_date = START_DATE.astimezone().isoformat()
events_result = service.events().list(
    calendarId='primary',  # Main calendar
    #calendarId='sjor2ciqns5rju6bd7u6tlis8k@group.calendar.google.com',  # Treino
    timeMin=from_date, # timeMax=to_date,
    maxResults=999, singleEvents=True,
    orderBy='startTime').execute()

events = events_result.get('items', [])

for e in events:
    try:
        descr = e['summary']
    except KeyError:
        continue
        
    if 'Tania' in descr or 'PT' in descr:
        start_time = e['start']['dateTime']
        # start 15min earlier to avoit back-to-back meetings
        start_time = f"{dt.datetime.fromisoformat(start_time) - dt.timedelta(minutes=15)}"
        # and convert to an outlook friendly format
        start_time = start_time[:start_time.find('+')].replace('T', ' ')
        print(e['summary'], start_time)
        if not DRY_RUN:
            OUTLOOK_CALENDAR.create_event(
                start=start_time,
                subject='pvt pt ssn',
                duration=105,
            )

Tania 2024-06-07 07:45:00
